# mBrain: headaches

In [1]:
from mobiledna.core.appevents import Appevents
import pandas as pd

In [2]:
ae = Appevents.load_data("./appevents.parquet")
df = ae.get_data()


2021-06-23 16:10:08 - Recognized file type as <parquet>.
2021-06-23 16:10:09 - 'load' took 0.214 seconds to complete.


In [3]:
ae.add_category(scrape=False).add_time_of_day()


Adding tod <startTime>: 100%|██████████| 15065/15065 [00:00<00:00, 1158930.15it/s]


# Variables

### Daily call duration & frequency

In [5]:
def calc_category_use(df: pd.DataFrame, categories: list, category_name: str, reverse= None, df_n: pd.DataFrame = None):
    """
    """
    logdays_pp = df.groupby("id")["startDate"].nunique()

    if reverse:
        mask = ~df["category"].isin(categories)
    else:
        mask = df["category"].isin(categories)

    duration = (df[mask].groupby(["id"])["duration"].sum() / logdays_pp / 60).rename(f"{category_name}_duration_min")
    frequency = (df[mask].groupby(["id"])["application"].count() / logdays_pp).rename(f"{category_name}_frequency")

    if df_n:
        if reverse:
            n_mask = ~df_n["category"].isin(categories)
        else:
            n_mask = df_n["category"].isin(categories)

        notifications = (df_n[n_mask].groupby("id")["application"].count() / logdays_pp).rename(f"{category_name}_notifications")
        res = pd.concat([duration, frequency, notifications], axis=1)

    else:
        res = pd.concat([duration, frequency], axis=1)

    return res

In [7]:
daily_call = calc_category_use(df=df, categories=["calling"], category_name="calling")
daily_call

,calling_duration_min,calling_frequency
id,,
0f17152a-ec9e-4105-b58d-25b2ae692388,0.389112,4.897436
a9ac43fd-cf95-40d5-9c9a-3582b88993fe,0.003523,0.054054


### Daily screentime
Calculated in `depression.ipynb`